In [12]:
import os
import pickle

use_def_prompt = os.getenv("USE_DEF_PROMPT", "False") == "True"
print("USE_DEF_PROMPT: ", use_def_prompt)

use_def_target = os.getenv("USE_DEF_TARGET", "False") == "True"
print("USE_DEF_TARGET: ", use_def_target)

use_number_target = os.getenv("USE_NUMBER_TARGET", "False") == "True"
print("USE_NUMBER_TARGET: ", use_number_target)

print("WARNING! USE_DEF_TARGET AND USE_NUMBER_TARGET MADE ONLY FOR HYPERNYMS PRED")


def clean_elem(elem, keys_to_remove_digits=["children"]):
    removes = set(keys_to_remove_digits)
    if not "changed" in elem.keys():
        for field in ["children", "parents", "grandparents", "brothers"]:
            if field in elem.keys():
                elem[field] = delete_techniqal(elem[field], remove=(field in removes))
                elem["changed"] = True
    return elem


def delete_techniqal(elem, remove):
    if isinstance(elem, str):
        if ((".n." in elem) or (".v." in elem)) and remove:
            return elem.split(".")[0].replace("_", " ")
        else:
            return elem.replace("_", " ")

    elif isinstance(elem, list):
        new_words = []
        for word in elem:
            new_words.append(delete_techniqal(word, remove))
        return new_words
def predict_parent_from_child(elem):
    """
    Predict the hypernym for the word “spaniel”. Answer: (sporting dog)
    """
    keys_to_remove_digits = []
    if use_def_prompt:
        keys_to_remove_digits.append(
            "children"
        )  # do not need numbers when provided definition
    if (not use_number_target) or (use_def_target):
        keys_to_remove_digits.append("parents")

    print(keys_to_remove_digits)
    clean = clean_elem(elem, keys_to_remove_digits=keys_to_remove_digits)

    if use_def_prompt:
        transformed_term = (
            "hyponym: " + clean["children"] + " (" + elem["child_def"] + ") | hypernym:"
        )
    else:
        transformed_term = "hyponym: " + clean["children"] + " | hypernym:"

    if use_def_target:
        target = clean["parents"] + " (" + elem["parent_def"] + ")" + ","
    else:
        target = clean["parents"] + ","

    return transformed_term, target

USE_DEF_PROMPT:  False
USE_DEF_TARGET:  False
USE_NUMBER_TARGET:  False
WARNING! USE_DEF_TARGET AND USE_NUMBER_TARGET MADE ONLY FOR HYPERNYMS PRED


In [2]:
with open('../data/test_wn_verbs.pickle', 'rb') as f:
    test = pickle.load(f)

In [13]:
from copy import deepcopy
elem = deepcopy(test[0])

In [14]:
predict_parent_from_child(elem)

['parents']


('hyponym: declassify.v.01 | hypernym:', 'free,')